In [1]:
from functools import reduce
import matplotlib.pyplot as plt
import numpy as np

from qiskit import BasicAer, ClassicalRegister, execute, QuantumCircuit, QuantumRegister
%matplotlib inline

# Algorithm

The class below encapsulates the $\alpha$-Quantum Phase Estimation algorithms for algorithms of the form:
$$\mathcal{A}|0\rangle^{\otimes n} = |\psi_0\rangle|0\rangle + |\psi_1\rangle|1\rangle$$
(hence the "binary" qualifier in the name of the class). The last qubit above in referred to as the "coin quantum register" in the code.

In [2]:
class QuantumBinaryBPE:
    def __init__(self, algo_qc, algo_qrs, coin_qr, ancilla_qr=None):
        """ Create a Quantum Bayesian Phase Estimation instance.
        
        Parameters
        ----------
        algo_qc: QuantumCircuit
            The qiskit circuit implementing the algorithm.
        algo_qrs: list(QuantumRegister)
            A list containing the quantum registers on which the algorithm operates.
        ancilla_qr: QuantumRegister
            An optional quantum register of ancillary qubits to perform the |0>^{\otimes n} flip.
        
        Returns
        -------
        """
        self.algo_qc = algo_qc
        self.algo_qrs = algo_qrs
        self.coin_qr = coin_qr
        self.algo_qubits = reduce(lambda x, y: x + y, [ list(algo_qr) for algo_qr in algo_qrs ], [])
        self.ancilla_qr = ancilla_qr
    
    def flip_0_circuit(self, control_qubit):
        """ Create a controlled quantum circuit that flips the |0>^{\otimes n} computational
        basis vector and leaves the other computational basis vectors unchanged.
        
        Parameters
        ----------
        control_qubit: Qubit
            The qubit controlling the circuit.
        
        Returns
        -------
        QuantumCircuit
            The controlled quantum circuit performing the operation described above.
        """
        qc = QuantumCircuit(*self.algo_qrs, control_qubit.register)
        for algo_qubit in self.algo_qubits:
            qc.x(algo_qubit)
        qc.h(self.algo_qubits[-1])
        if self.ancilla_qr:
            qc.mct([ control_qubit ] + self.algo_qubits[:-1], self.algo_qubits[-1], self.ancilla_qr, mode="basic")
        else:
            qc.mct([ control_qubit ] + self.algo_qubits[:-1], self.algo_qubits[-1], None, mode="noancilla")
        qc.h(self.algo_qubits[-1])
        for algo_qubit in self.algo_qubits:
            qc.x(algo_qubit)
        return qc
    
    def controlled_amplitude_estimation_unitary_circuit(self, control_qubit):
        """ Create a controlled quantum circuit implementing the "amplitude estimation unitary"
        
        Parameters
        ----------
        control_qubit: Qubit
            The qubit controlling the circuit.
            
        Returns
        -------
        QuantumCircuit
            A quantum circuit implementing the controlled amplitude estimation unitary. This unitary
            acts on a 2-dimensional Hilbert space with eigenvalues +/-theta, and the amplitude squared
            a to estimate satisfies a = sin(theta / 2)^2 with our conventions.
        """
        qc = QuantumCircuit(*self.algo_qrs, control_qubit.register)
        qc.x(coin_qr[0])
        qc.cz(control_qubit, coin_qr[0])
        qc.x(coin_qr[0])
        qc += self.algo_qc.inverse()
        qc += self.flip_0_circuit(control_qubit)
        qc += self.algo_qc
        return qc
    
    def partial_collapse_circuit(self):
        """ Create the 'partial collapse' (one can also say 'eigenstate amplification') circuit.
        
        Parameters
        ----------
        
        Returns
        -------
        tuple
            The first tuple element is a quantum circuit implementing the partial collapse
            circuit. This circuit consists of an RFPE circuit with parameters
            (M, theta) = (2, 0) followed by an RFPE circuit with parameters
            (M, theta) = (1, b * pi / 2) where b is the first measurement result. The second
            tuple element is the classical register containing the result of the first RFPE
            measurement. The third tuple element is the classical register containing the
            result of the second RFPE measurement.
        """
        control_qr = QuantumRegister(1, name="collapse_control")
        measurement1_cr = ClassicalRegister(1, "collapse_measurement_1")
        measurement2_cr = ClassicalRegister(1, "collapse_measurement_2")
        algo_circuit = self.controlled_amplitude_estimation_unitary_circuit(control_qr[0])
        qc = QuantumCircuit(*self.algo_qrs, control_qr, measurement1_cr, measurement2_cr)
        qc += self.algo_qc
        # RFPE with (M, theta) = (2, 0)
        qc.h(control_qr[0])
        qc += algo_circuit
        qc += algo_circuit
        qc.h(control_qr[0])
        qc.measure(control_qr[0], measurement1_cr)
        qc.x(control_qr[0]).c_if(measurement1_cr, 1)
        # RFPE with (M, theta) = (1, b * pi / 2) where b is the first RFPE measurement result
        qc.h(control_qr[0])
        qc.rz(-np.pi / 2, control_qr[0]).c_if(measurement1_cr, 1)
        qc += algo_circuit
        qc.h(control_qr[0])
        qc.measure(control_qr[0], measurement2_cr)
        return qc, measurement1_cr, measurement2_cr
    
    def elementary_phase_estimation_circuit(self, M, theta, measurement1_cr, measurement2_cr):
        """ Create an elementary RFPE circuit.
        
        Parameters
        ----------
        M: int
            The M parameter of the RFPE circuit (power to which the controlled unitary is raised).
        theta: float
            The theta parameter of the RFPE circuit (the Hadamarded control qubit undergoes
            Rz(-M * theta)).
        measurement1_cr: ClassicalRegister
            The classical register in which the first measurement of the partial collapse circuit
            is stored.
        measurement2_cr: ClassicalRegister
            The classical register in which the second measurement of the partial collapse circuit
            is stored.
        
        Returns
        -------
        QuantumCircuit
            The elementary RFPE circuit.
        """
        control_qr = QuantumRegister(1)
        control_cr = ClassicalRegister(1)
        algo_circuit = self.controlled_amplitude_estimation_unitary_circuit(control_qr[0])
        qc = QuantumCircuit(*self.algo_qrs, control_qr, control_cr, measurement1_cr, measurement2_cr)
        # U branch
        qc.h(control_qr[0]).c_if(measurement2_cr, 0)
        qc.rz(-M * theta, control_qr[0]).c_if(measurement2_cr, 0)
        for i in range(M):
            qc += algo_circuit
        qc.h(control_qr[0]).c_if(measurement2_cr, 0)
        qc.measure(control_qr, control_cr).c_if(measurement2_cr, 0)
        qc.x(control_qr[0]).c_if(control_cr, 1)
        # U dagger branch
        qc.h(control_qr[0]).c_if(measurement2_cr, 1)
        qc.rz(-M * theta, control_qr[0]).c_if(measurement2_cr, 1)
        for i in range(M):
            qc += algo_circuit.inverse()
        qc.h(control_qr[0]).c_if(measurement2_cr, 1)
        qc.measure(control_qr, control_cr).c_if(measurement2_cr, 1)
        return qc

    def estimation_step(self, rfpe_outcome, M, theta, mu, sigma, n_samples):
        """ Perform a single Bayesian update step.
        
        Parameters
        ----------
        M: int
            The power the unitary is raised to in the RFPE circuit.
        mu: float
            The mean of the prior phase distribution.
        sigma: float
            The standard deviation of the prior phase distribution.
        n_samples: int
            The number of phase samples to draw from the prior phase distribution in the Bayesian
            update step.
        
        Returns
        -------
        tuple
            A pair whose first element is the mean, second element the standard deviation of the
            posterior phase distribution.
        """
        # Sample phases from prior distribution and estimate the posterior one
        # by rejection sampling
        accepted_phis = []
        while len(accepted_phis) < n_samples:
            phi = min(np.pi / 2, max(0, mu + sigma * np.random.randn()))
            if np.random.uniform() \
                < 0.5 \
                + 0.25 * (-1) ** rfpe_outcome \
                    * (  \
                        (1.0 + np.sin(phi)) * np.cos(M * (phi - theta)) \
                        + (1.0 - np.sin(phi)) * np.cos(M * (phi + theta)) \
                    ):
                accepted_phis.append(phi)
        #print ("{} accepted samples".format(len(accepted_phis)))
        return (np.mean(accepted_phis), np.std(accepted_phis)) if len(accepted_phis) > 0 else (mu, sigma)

    def alpha_qpe(self, alpha, precision, n_samples_per_iteration, max_iterations, statistical_sampling_initialize=False, n_statistical_samples=0):
        """ Run the alpha-QPE algorithm and output estimated phase with uncertainty.
        
        Parameters
        ----------
        alpha: float
            The alpha parameter of the alpha-QPE algorithm.
        precision: float
            The target maximum value for the phase uncertainty. The algorithm stops once this level
            has been reached.
        n_initial_samples: int
            Number of samples to draw to obtain an estimate of the phase from classical Monte Carlo.
        n_samples_per_iteration: int
            The number of phase samples to draw at each Bayesian update step.
        max_iterations: int
            The maximum number of Bayesian update steps. The algorithm stops once this has been reached
            (even if the target precision has not).
        
        
        Returns
        -------
        tuple
            A pair whose first element is the estimated phase and second element the uncertainty
            on it.
        """
        if statistical_sampling_initialize:
            # Statistically sample the state to get an estimate of the phase.
            coin_cr = ClassicalRegister(1)
            qc = QuantumCircuit(coin_qr, coin_cr)
            qc += self.algo_qc
            qc.measure(coin_qr, coin_cr)
            counts = execute(qc, BasicAer.get_backend("qasm_simulator"), seed_simulator=np.random.randint(2 ** 32), shots=n_statistical_samples).result().get_counts()
            estimated_probability = counts.get("1", 0) / n_statistical_samples
            #print ("estimated probability (statistical sampling): {}".format(estimated_probability))
            estimated_phase = 2 * np.arcsin(np.sqrt(estimated_probability))
            #print ("estimated phase (statistical sampling): {}".format(estimated_phase))
            initial_sigma = 3 * np.sqrt(estimated_phase * (1 - estimated_phase) / n_statistical_samples) / np.sin(estimated_phase)
            if initial_sigma == 0:
                initial_sigma = np.pi
            #print ("initial prior distribution: ({}, {})".format(estimated_phase, initial_sigma))
            mu, sigma = estimated_phase, initial_sigma
        else:
            mu, sigma = np.pi / 2, np.pi
        # Iterate the RFPE algorithm. For each iteration, try to increase the component of one of the two
        # eigenvectors of the amplitude estimation unitary. If this succeeds, perform RFPE on the resulting
        # state (using either the amplitude estimation unitary or its dagger depending on which eigenvector
        # had their component increased).
        mu_history, sigma_history = [mu], [sigma]
        iterations = 0
        while iterations < max_iterations:
            #print ("iteration {}".format(iteration))
            M = int(1.154 / sigma) if alpha == 1 else int(1.0 / sigma ** alpha)
            collapse_qc, measurement1_cr, measurement2_cr = self.partial_collapse_circuit()
            step_theta = mu - sigma
            phase_estimation_qc = self.elementary_phase_estimation_circuit(M, step_theta, measurement1_cr, measurement2_cr)
            full_qc = collapse_qc + phase_estimation_qc
            counts = execute(full_qc, backend=BasicAer.get_backend("qasm_simulator"), seed_simulator=np.random.randint(2 ** 32), shots=1).result().get_counts(full_qc)
            result = list(counts.keys())[0]
            #print ("result: {}".format(result))
            if result[4] == "0":
                #print ("eigenstate amplification failed, skipping RFPE")
                continue
            mu, sigma = self.estimation_step(
                rfpe_outcome=int(result[0]),
                M=M,
                theta=step_theta,
                mu=mu,
                sigma=sigma,
                n_samples=n_samples_per_iteration
            )
            mu_history.append(mu)
            sigma_history.append(sigma)
            if sigma < precision:
                break
            print ("iteration {}: posterior estimate: ({}, {})".format(iterations, mu, sigma))
            iterations += 1
        return mu, sigma, mu_history, sigma_history

# Test

In this test, the state prepared by the algorithm is simply $\cos\frac{\theta}{2}|0\rangle + \sin\frac{\theta}{2}|1\rangle$ and the phase to be estimated is then $\theta$.

In [3]:
theta = np.pi / 5

In [4]:
precision = 1e-5
max_iterations = 60
n_samples_per_iteration = 1000

In [5]:
coin_qr = QuantumRegister(1)
qc = QuantumCircuit(coin_qr)
qc.ry(theta, coin_qr[0])

In [6]:
bpe = QuantumBinaryBPE(algo_qc=qc, algo_qrs=[coin_qr], coin_qr=coin_qr)

## $\alpha = 1$

In [ ]:
mu, sigma, mu_history, sigma_history = bpe.alpha_qpe(
    alpha=0.6,
    precision=precision,
    n_samples_per_iteration=n_samples_per_iteration,
    max_iterations=5000,#max_iterations,
    statistical_sampling_initialize=False,#True,
    n_statistical_samples=min(65536, int(1 / precision)),
)

iteration 0: posterior estimate: (0.9551310621014415, 0.7171144805072562)
iteration 1: posterior estimate: (0.8255129869586285, 0.5281844613966836)
iteration 2: posterior estimate: (0.75063207734208, 0.45083381705129955)
iteration 3: posterior estimate: (0.7255309221626485, 0.410331569713375)
iteration 4: posterior estimate: (0.6964408576569459, 0.38840697565437615)
iteration 5: posterior estimate: (0.6933853646160141, 0.3499358683694272)
iteration 6: posterior estimate: (0.6602984219858001, 0.3256527380774275)
iteration 7: posterior estimate: (0.649693175307768, 0.3286500929692238)
iteration 8: posterior estimate: (0.6475755571050049, 0.31576002746992865)
iteration 9: posterior estimate: (0.641640058564335, 0.30007056980108804)
iteration 10: posterior estimate: (0.57178173282024, 0.25937881696595044)
iteration 11: posterior estimate: (0.5474867843741851, 0.2457423969139245)
iteration 12: posterior estimate: (0.5162605778678567, 0.2377527935612557)
iteration 13: posterior estimate: (0.

iteration 110: posterior estimate: (0.48899186773073633, 0.11645771008169763)
iteration 111: posterior estimate: (0.49543741750252074, 0.12686502335921376)
iteration 112: posterior estimate: (0.4906508209873145, 0.12079144138687302)
iteration 113: posterior estimate: (0.5026121902196647, 0.12680786838710945)
iteration 114: posterior estimate: (0.5130519963417515, 0.13265694854455956)
iteration 115: posterior estimate: (0.5122939988089953, 0.12741197687835734)
iteration 116: posterior estimate: (0.5285926351090797, 0.13460982400138155)
iteration 117: posterior estimate: (0.5209877028155281, 0.1282447603676993)
iteration 118: posterior estimate: (0.5176158613583927, 0.12580529720496303)
iteration 119: posterior estimate: (0.5276177612844322, 0.13588873138921675)
iteration 120: posterior estimate: (0.5426551725518685, 0.14520384292662133)
iteration 121: posterior estimate: (0.5305928026537695, 0.1378640812209252)
iteration 122: posterior estimate: (0.5172270098650056, 0.13354893448622202)

iteration 217: posterior estimate: (0.6020997792094313, 0.06027799222012722)
iteration 218: posterior estimate: (0.6030037367212785, 0.06075644208685546)
iteration 219: posterior estimate: (0.6006084974174574, 0.056506151610649244)
iteration 220: posterior estimate: (0.6009117628485731, 0.05523679864326948)
iteration 221: posterior estimate: (0.5995884199855108, 0.05447914059484661)
iteration 222: posterior estimate: (0.5994155545105968, 0.053373225973634295)
iteration 223: posterior estimate: (0.5983899782807758, 0.0527133475323496)
iteration 224: posterior estimate: (0.5995924147474471, 0.050232432773177155)
iteration 225: posterior estimate: (0.5964101205591759, 0.04938963810187978)
iteration 226: posterior estimate: (0.5944839010838777, 0.047493462597842845)
iteration 227: posterior estimate: (0.5906768528274808, 0.04665556142334488)
iteration 228: posterior estimate: (0.5913031684476334, 0.044838132747444144)
iteration 229: posterior estimate: (0.5879042659303504, 0.04340209558901

iteration 323: posterior estimate: (0.6067093893290271, 0.028756903694648457)
iteration 324: posterior estimate: (0.6056488463800772, 0.028412605471729514)
iteration 325: posterior estimate: (0.605385927060825, 0.027565304978905067)
iteration 326: posterior estimate: (0.6037459657334413, 0.028056180110785173)
iteration 327: posterior estimate: (0.6015259623375271, 0.02788854064427552)
iteration 328: posterior estimate: (0.6012481979671324, 0.02685978936523813)
iteration 329: posterior estimate: (0.6028493760959257, 0.026717445392192927)
iteration 330: posterior estimate: (0.6012930539883942, 0.025550387237802816)
iteration 331: posterior estimate: (0.6023379668438615, 0.025786819987082037)
iteration 332: posterior estimate: (0.6019864289737878, 0.025942232725949785)
iteration 333: posterior estimate: (0.6018607104265042, 0.02603536243207169)
iteration 334: posterior estimate: (0.6012157950081541, 0.025676343398568016)
iteration 335: posterior estimate: (0.602907051275483, 0.02497654149

iteration 429: posterior estimate: (0.6125359433214118, 0.019993708330932267)
iteration 430: posterior estimate: (0.6117725384549424, 0.020149359192743726)
iteration 431: posterior estimate: (0.6110387423469082, 0.020421064033188448)
iteration 432: posterior estimate: (0.6104527696046123, 0.020935090533996017)
iteration 433: posterior estimate: (0.610265057349181, 0.020601183517003435)
iteration 434: posterior estimate: (0.6105886203909514, 0.020478939413190487)
iteration 435: posterior estimate: (0.6103815199376413, 0.0199034260228137)
iteration 436: posterior estimate: (0.6110296139932918, 0.019893453144041104)
iteration 437: posterior estimate: (0.6109250048193557, 0.01966894061142545)
iteration 438: posterior estimate: (0.6110602365800893, 0.01931695453453346)
iteration 439: posterior estimate: (0.6114735094875922, 0.01976868711090308)
iteration 440: posterior estimate: (0.6119097208658291, 0.020200118233206662)
iteration 441: posterior estimate: (0.6119339591570059, 0.019513005769

...

iteration 3535: posterior estimate: (0.6255804482873972, 0.0002392175895333904)
iteration 3536: posterior estimate: (0.6255792709630144, 0.00023566168345832346)
iteration 3537: posterior estimate: (0.6255824330067932, 0.00023507404028461816)
iteration 3538: posterior estimate: (0.6255858220820371, 0.00023751804558095523)
iteration 3539: posterior estimate: (0.6255913465894926, 0.00022876700130312566)
iteration 3540: posterior estimate: (0.6256026320429591, 0.00022152028080832154)
iteration 3541: posterior estimate: (0.6255976609905872, 0.000225076302221784)
iteration 3542: posterior estimate: (0.6255957298487427, 0.0002220975828256787)
iteration 3543: posterior estimate: (0.6256035965673536, 0.00022568056660058798)
iteration 3544: posterior estimate: (0.6255962052666587, 0.00021859206946675286)
iteration 3545: posterior estimate: (0.6255944905142341, 0.0002142168118429461)
iteration 3546: posterior estimate: (0.6255877378866919, 0.00020638840116575657)
iteration 3547: posterior estimat

iteration 3637: posterior estimate: (0.6256292937827163, 0.00022444511133716712)
iteration 3638: posterior estimate: (0.6256334376546827, 0.0002252495467649924)
iteration 3639: posterior estimate: (0.62563197808644, 0.00023224378405132162)
iteration 3640: posterior estimate: (0.6256939086480453, 0.00023082101927261953)
iteration 3641: posterior estimate: (0.6256940714790605, 0.00022914620127394448)
iteration 3642: posterior estimate: (0.6256977207599521, 0.00023638896790548173)
iteration 3643: posterior estimate: (0.625697488883053, 0.00023042642748227302)
iteration 3644: posterior estimate: (0.6256911915136366, 0.00023020168099032105)
iteration 3645: posterior estimate: (0.6256968836127323, 0.0002286159746714512)
iteration 3646: posterior estimate: (0.6257008299545173, 0.0002263716385035392)
iteration 3647: posterior estimate: (0.625698822161897, 0.00023036288052798466)
iteration 3648: posterior estimate: (0.6257032648186305, 0.00022317513133828076)
iteration 3649: posterior estimate:

iteration 3739: posterior estimate: (0.6257530783409401, 0.00014792303055108217)
iteration 3740: posterior estimate: (0.6257539102539708, 0.00014997169902326434)
iteration 3741: posterior estimate: (0.6257622693873627, 0.00014264181580770787)
iteration 3742: posterior estimate: (0.6257635322984777, 0.00014263590817715943)
iteration 3743: posterior estimate: (0.6257592809171775, 0.00014016789517660378)
iteration 3744: posterior estimate: (0.6257623467834215, 0.0001364012378858914)
iteration 3745: posterior estimate: (0.625770495854325, 0.00013536771225609013)
iteration 3746: posterior estimate: (0.6257692751425088, 0.0001379715931399562)
iteration 3747: posterior estimate: (0.625773157646066, 0.00013625967066570557)
iteration 3748: posterior estimate: (0.6257623259161651, 0.0001356875007710967)
iteration 3749: posterior estimate: (0.625755603809269, 0.0001330074050574089)
iteration 3750: posterior estimate: (0.6257539800152313, 0.00013409156490121275)
iteration 3751: posterior estimate:

iteration 3841: posterior estimate: (0.6259232534782909, 0.00014324310353459933)
iteration 3842: posterior estimate: (0.6259184636512614, 0.00014469889482305212)
iteration 3843: posterior estimate: (0.6259177786830882, 0.00014794161348895502)
iteration 3844: posterior estimate: (0.6259147129549492, 0.0001467702905225472)
iteration 3845: posterior estimate: (0.6259046931787506, 0.0001455120251447013)
iteration 3846: posterior estimate: (0.6259117751282155, 0.00014504653539731163)
iteration 3847: posterior estimate: (0.6259197616873421, 0.0001428519867417933)
iteration 3848: posterior estimate: (0.6259180491640105, 0.00013646626628895274)
iteration 3849: posterior estimate: (0.6259192989807023, 0.00013720316023214357)
iteration 3850: posterior estimate: (0.6259214922219143, 0.0001366065950948105)
iteration 3851: posterior estimate: (0.625919650892275, 0.00013544154465234203)
iteration 3852: posterior estimate: (0.6259152321415824, 0.00012854778358051714)
iteration 3853: posterior estimat

iteration 3943: posterior estimate: (0.6259737813768135, 9.931710058805253e-05)
iteration 3944: posterior estimate: (0.6259748201247376, 9.884679045081378e-05)
iteration 3945: posterior estimate: (0.6259755585036566, 9.752040748943866e-05)
iteration 3946: posterior estimate: (0.6259746573162277, 9.61503927325025e-05)
iteration 3947: posterior estimate: (0.6259794648432936, 9.273465446683447e-05)
iteration 3948: posterior estimate: (0.6259810239547394, 9.30418228122177e-05)
iteration 3949: posterior estimate: (0.6259764419757967, 9.854017655956798e-05)
iteration 3950: posterior estimate: (0.6259785200869863, 9.75906194402039e-05)
iteration 3951: posterior estimate: (0.6259787816327103, 9.83558909524352e-05)
iteration 3952: posterior estimate: (0.6259824693887132, 9.966041373094873e-05)
iteration 3953: posterior estimate: (0.6259868894127375, 0.00010086915991766608)
iteration 3954: posterior estimate: (0.6259895244203407, 9.417054784185927e-05)
iteration 3955: posterior estimate: (0.6259

In [ ]:
plt.plot(np.log(np.array(sigma_history[:60]) / sigma_history[0]), "+")